In [7]:
import os
import sys
import json
from pathlib import Path
sys.path.append(os.path.abspath('./src'))

import pandas as pd

In [8]:
submission_name = "FasterCnnByteTracking"

submission_name = submission_name + ".csv"

In [9]:
import ast

submission_dir = Path("submissions")
submission_dir.mkdir(exist_ok=True, parents=True)


submission_file = pd.read_csv("submission_file.csv")
submission_file['objects'] = submission_file['objects'].apply(lambda x: ast.literal_eval(x))

assert isinstance(submission_file['objects'].iloc[0], list), "The first 'objects' entry is not a list!"

In [10]:
df_tracking = pd.read_csv("fasterrcnn_batch8_freeze_test_th07.csv", header=None)
df_tracking.columns = ["frame", "tracked_id", "x", "y", "w", "h", "confidence", "class"]

df_group = df_tracking.groupby('frame')[["tracked_id", "x", "y", "w", "h", "confidence"]].apply(
    lambda g: g.to_dict(orient="records")
)

In [11]:
submission_file.loc[submission_file["objective"] == "tracking", "objects"] = submission_file["frame"].map(df_group)
# submission_file["objects"] = submission_file["objects"].apply(lambda x: x if isinstance(x, list) else [])

In [12]:
submission_file.to_csv(submission_dir / submission_name ,index=None)